#Data Cleanup
* find missing lats and longs
* find duplicates
* investigate unexpected lats and longs (some values appear not to be in Toronto)

In [1]:

from sqlalchemy.orm import sessionmaker
from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopy.distance
import geocoder
import os
from dotenv import load_dotenv, find_dotenv
import time
DEBUG=0


In [2]:
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename='../logs/datacleanup.log',level=logging.DEBUG)


In [3]:
def get_key(env='BING'):
    load_dotenv(find_dotenv())
    DEBUG = os.getenv("DEBUG")
    # load environment variables
    if env=='BING':
        KEY = os.getenv("BING_KEY")
    else:
        KEY = os.getenv("GOOGLE_KEY")
    #DEBUG = os.getenv("DEBUG")
    return KEY

In [20]:
def get_lats_longs(address):
    '''This function uses GeoCoder to retrieve the latitude and longitude for neighbourhoods.
    address to look up
    returns lat and long
    '''
    KEY = get_key("GOOGLE_KEY")
    # # get geocode for postal codes
    # initialize your variable to None
    lat_lng_coords = None
    lat=None
    long=None
    max_loops = 10
    curr_loop =0
    # loop until you get the coordinates (sometime dont' get a response)
    lat_lng_coords = None
    while(lat_lng_coords is None and curr_loop < max_loops):
      # g = geocoder.bing('{}'.format(address),key=KEY)
        g = geocoder.google('{}'.format(address), key=KEY)
       # g= geocoder.google('{}'.format(address))
        if DEBUG > 0:
            print (g)
        lat_lng_coords = g.latlng
        if DEBUG > 0:
            print(lat_lng_coords)
        if lat_lng_coords == None:
            time.sleep(1)
        curr_loop += 1
    if lat_lng_coords is not None:
        lat=lat_lng_coords[0]
        long=lat_lng_coords[1]

    return lat,long

In [5]:
def cleanup_address(row):
   
    lat=None
    long=None
    row.print_me()
    if a.address[0] == '0': 
        # look for "0 <street" format -- usually first digit
        print ("street address is zeroed")
        new_search = row.name + " " +  a.address[1:]  + ", Toronto ON"
        print(f'try searching on: {new_search}')
        lat, long=get_lats_longs(new_search)
    elif "Toronto" not in a.address:
         # check if missing Toronto and try adding it
        print('address is missing toronto')
        row.address = row.address +  ", Toronto ON"
        lat, long=get_lats_longs(row.address)
    print(f'new lat {lat} and long{long}')

    return lat, long

In [6]:
def update_missing_coords():
    '''
    Find all rows in PointsofINterest table that are missing lat and long
    Lookup address using Geocoder and update database row
    '''
    null_lats = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None)
#     row = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None).first()
#     lat, long= get_lats_longs(row.address)
#     props = {'latitude':lat, 'longitude':long}

#     for key, value in props.items():
#         setattr(row, key, value)

    for row in null_lats:
        #null_lat = session.query(PointsOfInterest).get(1)
        lat, long= get_lats_longs(row.address)
        props = {'latitude':lat, 'longitude':long}

        for key, value in props.items():
            setattr(row, key, value)

    # save up our commits
    session.commit()
    session.flush()
    return True


In [7]:
def check_missing_coords():
    num_missing_coords = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None).count()
    print(f"There are {num_missing_coords} records missing lat/long coordindates.")
    if num_missing_coords> 0:
        update_missing_coords()

In [8]:
# db=connect_db() #establish connection / creates database on first run
# Session = sessionmaker(bind=db)
# session = Session()

# #check_missing_coords()
# df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
# df.sort_values('name', inplace=True)

In [9]:
#session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None).count()

0

In [10]:
#check_missing_coords()

In [11]:
# df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
# df.sort_values('poi_id', ascending=True, inplace=True)

In [ ]:
def delete_poi_by_id(id):
    poi = session.query(PointsOfInterest).filter(PointsOfInterest.poi_id==id).first()
    print(f"deleting {poi.name}")
    session.delete(poi)
    session.commit()
    return True

In [13]:

# df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
# df.sort_values('name', inplace=True)
# df.head()

,poi_id,name,build_year,demolished_year,address,latitude,longitude,external_url,image_url,heritage_status,current_use,poi_type,source,details
867,830,51 Madison Avenue,1902,None,51 Madison Avenue The Annex Toronto,43.669880,-79.403490,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,"West Annex Phase I, Madison Avenue, Heritage C...",Residential,Detached house,http://www.acotoronto.ca/,This property contributes to the Toronto Annex...
3314,2180,"McKinsey & Company, Toronto Headquarters",1999,None,110 Charles Street West University of Toronto ...,43.668010,-79.390990,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Heritage property,Office,Low-rise,http://www.acotoronto.ca/,The building was designated a heritage propert...
3326,2192,Oben Flats | Harbord Towns,2015,None,354 Harbord Street Little Italy Toronto,43.659430,-79.421020,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,No heritage status,Residential,Low-rise,http://www.acotoronto.ca/,None
2142,8398,"""Ned"" Hanlan 1855-1908",None,None,None,43.627595,-79.389449,http://torontoplaques.com/Pages/Ned_Hanlan.html,None,None,None,Plaque,http://torontoplaques.com,"So why is there a monument to ""Ned"" Hanlan on ..."
1569,7839,"""Old"" City Hall",None,None,None,43.652290,-79.381100,http://torontoplaques.com/Pages/Old_City_Hall....,None,None,None,Plaque,http://torontoplaques.com,When the Eaton Centre was being planning back ...


In [14]:
def find_and_delete_dups(df):
    df.sort_values('name', inplace=True)
    prev_name=''
    prev_addr = ''
    prev_src =  ''
    prv_exturl=''
    for ix, row in df.iterrows():
        if prev_name == row['name'] and prev_addr == row['address'] and  prev_src==row['source'] and prv_exturl==row['external_url']:
            print(f"duplicate {prev_name} at {prev_addr} from {prev_src} and {prv_exturl}")
            delete_poi_by_id(row['poi_id'])
            #print(row)
        prev_name = row['name']
        prev_addr = row['address']
        prev_src = row['source']
        prv_exturl=row['external_url']

# Try to find points outside Toronto

In [15]:
# #measure distance from Yonge/Dundas square
# starting_lat =43.656287
# starting_long= -79.380898


In [16]:
def update_coords(id):
    poi_to_update = session.query(PointsOfInterest).filter(PointsOfInterest.poi_id == id).first()
    print(f"Checking coordinates for{poi_to_update.name}. Current coordinates for {poi_to_update.address} are {poi_to_update.latitude}, {poi_to_update.longitude}")
    logging.debug(f"Checking coordinates for {id} {poi_to_update.name}. Current coordinates for {poi_to_update.address} are {poi_to_update.latitude}, {poi_to_update.longitude}")
    
    lat, long = get_lats_longs(poi_to_update.address)
    logging.debug(f"Updating to {lat},{long}")
    
    poi_to_update.latitude = lat
    poi_to_update.longitude =long

    session.commit()
    session.flush()

    print(lat, long)

In [17]:
def find_dist(df, lat, long):
    avail_points = []
    
    for ix, row in df.iterrows():
        curr_pt = geopy.distance.geodesic((row['latitude'], row['longitude']), (lat, long)).km
        print(curr_pt)
        
        avail_points.append(curr_pt)
    df['dist_start'] = avail_points
    return df

In [22]:
def find_points_outside_TO(df, dist=50, starting_lat=43.656287,starting_long= -79.380898):
    ''' Find points more than 50KM from downtown Toronto (Yonge Dundas Square is default) and try to update'''
    df=find_dist(df, starting_lat, starting_long)
    for ix, row in df[df['dist_start']>dist].iterrows():
        update_coords(row['poi_id'])

2.3667369756455967
1.5359402674244518
3.2552521876925558
3.2616421042764583
0.444387532346308
1.9195309832306464
3.6412878041242593
9.090393173317475
0.5665543969033993
1.7628033720159686
9.877872958775013
2.2883444636278596
2.0813462168978414
3.207305223214778
1.7628033720159686
2.46438525992739
4.373267961287335
8.116601724400397
11.415013542885271
6.047856206915685
1.7753962597029656
3.221828886287939
3.221828886287939
3.221828886287939
3.221828886287939
1.808194522046581
4.131872200846019
1.7701123267001884
1.8057049583431195
1.693856809410477
2.0681170284662787
5561.355812721815
3.221828886287939
1.6855127371452718
3.221828886287939
3.221828886287939
4.228098670954927
1.6973764855256779
2.3638452375270966
2.457130618259844
2.0577619267679457
10.835932348951262
7.220363412319195
13.544851116913557
7.219588169374345
1.3796596317411594
3.122645169843531
3.122645169843531
3.122645169843531
3.122645169843531
3.122645169843531
3.122645169843531
10.061242865788191
1.781610053932463
3.663

55.97574583432078
1.229320285801917
5589.795291732281
5556.868736854073
2.2600973459637506
2.3835745850272128
1.6691014716139274
1.77387093986477
1.3370717315735456
12.007406980690934
9.86201886332835
4.002275223681157
8.308316145796175
1.0928762221943977
0.920147891249827
6.322113474853385
0.7184080172757127
8.532569035099613
5626.993033523212
5626.993033523212
2.1310177929617886
2.1064428026765505
5588.431707053247
581.1751655241753
3.696214726064494
1.7882701278174393
5588.431707053247
1.442624411439898
2.3515702186912217
3.258682150933944
11.59756967989794
1.576623498781832
1.8770408221262116
1.716215045380089
1.7066147579196695
1.9270283383983164
3.221828886287939
4.101782913579131
2.2277303394889167
1.0615322502307574
0.920147891249827
1.3695585954721685
6.463929064798408
1.4789638486556629
0.920147891249827
1.4992312133498287
4.717555619308345
2.6312832853064885
0.9484893753292152
2.1622374797830335
9.809643849304418
581.1751655241753
7.311237347319074
2.266004043629062
1.995189

1.174054991936644
1.3090162951166497
3.221828886287939
1.7428460998132624
2.0624347657704583
9.649687779100269
9.461377168358736
3.0661289695626657
7.567594728655298
2.4482331940056223
1.12401790847866
2.598902928946846
1.0688450298576118
2.953593147315304
2.4334912668340736
2.599257704254683
2.4173819096373848
2.9399790793201634
8.448671252054112
2.625132128391527
2.9634224472815482
2.440371315575655
2.448715945391877
2.4520749810330016
2.4537250870399734
1.8437094267848806
1.9437900591144923
1.430146122836426
2.4242688311121294
2.9501994631555335
3.203141745486664
1.3767270988090723
7.579235751324318
2.4621599142083994
1.7992699216322006
2.9605637402030127
9.720463339890165
2.428382777388519
2.961721577591865
1.6101530011391918
1.5049081232349086
6.302703517317595
1.802883315577084
1.5294340960919819
0.48085110343636533
7.5917805409612695
2.4675786978812373
1.2908785658127602
1.8465178349417202
1.5497855961327518
2.4173526554293074
0.7187122738203756
1.7117619189550897
2.438065934403

6.926461291692202
5.589886277522887
5.617120322495698
5.625508906221129
0.7396635658702403
0.7351968534500419
5.187824772605289
2.7595289520096604
3.66813795112139
0.6471959327459165
6.935522317805219
1.0807386007261528
1.7128580422059692
0.8534088254607626
1.0459893989234808
2.2492241400241566
10.80825885851295
0.6473005286022244
56.08941481976597
0.6284313478264637
1.0806445644208424
1.0798659018149401
2.871139754588796
4.384244799689096
6.951108824995851
4.466782792119191
1.3419305824933936
0.9798818138323343
6.808755037532591
0.6666908903751738
1.1738409343326617
4.371972221902098
1.4737896233872851
5.828218913288011
1.7413533090863567
5.187824772605289
1.950547776737291
5.915783186166955
12.073524251362775
2.2680328883234253
3.6638103527740022
12.200796467818547
22.460788977192706
4.079732215642598
6.253760120089888
6.161654658089667
4.017768572948415
1.3035605266731645
7.538605057932361
7.087796451473196
1.3963820027135554
5.84220859534207
3.3456027573889537
3.968923307811064
3.5

6.86414569831144
4.079732215642598
13.236416589808329
0.9261272732738938
3.3052964206473847
4.198051349794429
4.025714923015336
9.214327215748924
3.8983539369930504
3.7897057474428455
3.701117308718938
3.3052964206473847
4.567209870790081
3.3052964206473847
4.183000050248928
3.3052964206473847
1.9993040888319515
0.7238875420101438
7.686953023055812
5561.355812721815
7.686953023055812
5588.431707053247
5588.431707053247
1.6501417331273407
3.391423508098698
1.5739937503835193
19.200695400307993
12.881441813799151
13.007179111699926
10.692192192811182
12.735980954071213
1.0519416981720733
5.400644019058123
10.920324180155152
1.0209722919699893
14.035627277760257
14.21736062571977
0.743083343464414
6.404145970604743
0.837498610982972
2.251323969215156
1.1796746622840415
56.34458860285358
0.8274336398075185
12.733395116425477
2.990144044019316
5.863982443057143
12.744258658887368
1.0204296070262224
1.0220421903120585
0.5079267332921443
1.0897370943593936
1410.1445100272751
3.744347389284830

...

43.784594 -79.4407647
Checking coordinates forFrancis Libermann Catholic High School. Current coordinates for 4640 Finch Avenue East Agincourt Scarborough are 54.23634, -0.36828
43.8070281 -79.27418469999999
Checking coordinates forFrank Oke Secondary School. Current coordinates for 500 Alliance Avenue Mount Dennis York are 53.9533309936523, -1.08342003822327
43.6774894 -79.4885718
Checking coordinates forGO Milliken. Current coordinates for 39 Red Lea Avenue Steeles Scarborough are 54.2778282165527, -0.410679996013641
43.822085 -79.30368519999999
Checking coordinates forGalloway Road Public School. Current coordinates for 192 Galloway Road Guildwood Scarborough are 54.2778282165527, -0.410679996013641
43.7647888 -79.1964249
Checking coordinates forGeorge B. Little Public School. Current coordinates for 125 Orton Park  Woburn Scarborough are 42.4792213439941, -71.1506118774414
43.7717396 -79.2085771
Checking coordinates forGeorge Harvey Collegiate Institute. Current coordinates for 170

43.6631967 -79.4868796
Checking coordinates forKingsquare. Current coordinates for 33 King Street Weston York are 53.95723, -1.08249
43.7021625 -79.51876419999999
Checking coordinates forKnob Hill Public School. Current coordinates for 25 Seminole Avenue Eglinton Scarborough are 54.2778282165527, -0.410679996013641
43.7455895 -79.2493198
Checking coordinates forKnox United Church. Current coordinates for 4156 Sheppard Avenue East Agincourt Scarborough are 54.2778282165527, -0.410679996013641
43.7852746 -79.2813627
Checking coordinates forLCBO Store. Current coordinates for 510 Brimley Avenue Eglinton Scarborough are 54.2778282165527, -0.410679996013641
43.7355993 -79.2481827
Checking coordinates forLambton Park Community School. Current coordinates for 50 Bernice Crescent Lambton York are 53.9533309936523, -1.08342003822327
43.6673783 -79.4982237
Checking coordinates forLittle York Inn. Current coordinates for 187 King Street East Downtown East Toronto are 43.16172, -79.24378
43.650636

43.72191 -79.2380938
Checking coordinates forR.L. Clark Water Treatment Plant. Current coordinates for 1 Twenty Third Street Long Branch Etobicoke are 41.1526557105705, -79.0800154692835
43.5931507 -79.5192476
Checking coordinates forRainbow Village. Current coordinates for 2466 Eglinton Avenue East Eglinton Scarborough are 54.2778282165527, -0.410679996013641
43.7344 -79.2609379
Checking coordinates forRainbow Village. Current coordinates for 2460 Eglinton Avenue East Eglinton Scarborough are 54.2778282165527, -0.410679996013641
43.7342567 -79.26289489999999
Checking coordinates forRobert Service Senior Public School. Current coordinates for 945 Danforth Road Kennedy Park Scarborough are 54.2778282165527, -0.410679996013641
43.7290486 -79.2511939
Checking coordinates forRockford Public School. Current coordinates for 60 Rockford Road  Branson North York are 36.6415596008301, -93.217887878418
43.78582189999999 -79.45236919999999
Checking coordinates forRoselands Junior Public School. C

43.8124993 -79.2266994
Checking coordinates forSt. Henry Catholic School. Current coordinates for 100 Bamburgh Circle Steeles Scarborough are 55.6075782775879, -1.71634995937347
43.8138055 -79.3279841
Checking coordinates forSt. James' Cathedral War Memorial. Current coordinates for 106 King Street East Downtown East Toronto are 43.15873, -79.24565
43.65032859999999 -79.3737598
Checking coordinates forSt. James´ Cathedral. Current coordinates for 106 King Street East Downtown East Toronto are 43.15873, -79.24565
43.65032859999999 -79.3737598
Checking coordinates forSt. John´s United Church. Current coordinates for 2 Nobert Road Sullivan Scarborough are 54.2778282165527, -0.410679996013641
43.7718912 -79.3197533
Checking coordinates forSt. Lawrence Galleries. Current coordinates for 142 King Street East Downtown East Toronto are 43.1597733, -79.2447525
43.6505372 -79.3722913
Checking coordinates forSt. Lawrence Hall. Current coordinates for 151 King Street East Downtown East Toronto are

43.7467198 -79.2905215
Checking coordinates forWexford Presbyterian Church. Current coordinates for 7 Elinor Avenue Wexford Scarborough are 54.2778282165527, -0.410679996013641
43.7446257 -79.30047189999999
Checking coordinates forWexford Residence. Current coordinates for 1860 Lawrence Avenue East Wexford Scarborough are 54.2778282165527, -0.410679996013641
43.7439584 -79.3024504
Checking coordinates forWhite Oak Place. Current coordinates for 665 Kennedy Road Kennedy Park Scarborough are 54.2778282165527, -0.410679996013641
43.7260326 -79.26427559999999
Checking coordinates forWigwamen Housing. Current coordinates for 20 Sewells Road Malvern Scarborough are 52.0932456, -2.3343607
43.80764569999999 -79.2172259
Checking coordinates forWilliam G. Miller Public School. Current coordinates for 60 Bennet Road West Hill Scarborough are 54.2778282165527, -0.410679996013641
43.7685951 -79.1650385
Checking coordinates forWilliam Lyon Mackenzie House. Current coordinates for 82 Bond Street Down

In [ ]:
# df.sort_values('dist_start', ascending=False, inplace=True)
# df.head(10)
# df.loc[0,:]

In [ ]:
# for ix, row in df[df['dist_start']>50].iterrows():
#     update_coords(row['poi_id'])

In [21]:
#update_coords(2335)

Checking coordinates forToronto Free Presbyterian Church. Current coordinates for 5810 Finch Avenue East Malvern Scarborough are 43.8163204, -79.2276371
43.8163204 -79.2276371


In [ ]:
db=connect_db() #establish connection / creates database on first run
Session = sessionmaker(bind=db)
session = Session()

check_missing_coords()

# remove duplicate rows
df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
df.sort_values('name', inplace=True)
find_and_delete_dups(df)

# get fresh df from database with removed deletes
df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
df.sort_values('poi_id', inplace=True)
find_points_outside_TO(df)